<a href="https://colab.research.google.com/github/george-birchenough/onedot/blob/main/Onedot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
import pandas as pd
import os 
import numpy as np
import scipy as scipy
import json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ln -s '/content/drive/My Drive' /mydrive

Mounted at /content/drive


In [ ]:
json_path = "/mydrive/supplier_car.json"

In [ ]:
# Load the data from the .json file. The file is comprised of multiple records, so we need to load them individually. It takes 2-3 mins. 
input_df = pd.DataFrame()
for line in open(json_path, 'r'):
    record_dict = json.loads(line)
    temp_df = pd.DataFrame.from_dict(record_dict, orient='index').transpose()
    input_df=input_df.append(temp_df)
df_backup = input_df.copy()
input_df.head()

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,Attribute Names,Attribute Values,entity_id
0,976.0,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,Seats,2,0001fda6-192b-46a8-bc08-0e833f904eed
0,1059.0,MERCEDES-BENZ,ML 350 Inspiration,MERCEDES-BENZ ML 350 Inspiration,ML 350,ML 350 Inspiration,Hp,235,00107c2d-0071-4475-88f0-810133638b7e
0,524.0,AUDI,S6 Avant quattro 4.2,AUDI S6 Avant quattro 4.2,S6,S6 Avant quattro 4.2,FuelTypeText,Benzin,00126794-a8ef-48fe-93d6-43cfc69fbfb6
0,608.0,SAAB,9-3 2.0i-16 TS Aero,SAAB 9-3 2.0i-16 TS Aero,9-3,9-3 2.0i-16 TS Aero,Ccm,1985,00182529-1bf7-4f93-89fa-2e8e634b2c9d
0,726.0,PORSCHE,911 Turbo Cabrio,PORSCHE 911 Turbo Cabrio,911,911 Turbo Cabrio,BodyColorText,schwarz mét.,002d30c2-43f6-4905-868f-160dbc445c56


In [ ]:
# Looking at the data, we need to expand the attribue names/values to create a set of new columns, and then make 1 row for each 'ID'. We can drop the 'entity_id'. 
input_df = df_backup.copy()
pivot_df = input_df.pivot(index = 'ID', columns = 'Attribute Names', values = 'Attribute Values')
input_df.drop( ['Attribute Names', 'Attribute Values', 'entity_id'] , axis = 1, inplace = True)
input_df = input_df.drop_duplicates()
input_df.set_index('ID', inplace=True)
pre_pro_df = pd.concat([input_df, pivot_df], axis = 1).reset_index(drop=True)
normal_df = pre_pro_df.copy() 

In [ ]:
# Normalising the Car types with some assumptions made according to wikipedia definitions etc, avoiding inaccuracy. 
# There is some information lost as the range of input values != the range of target values. Some input values must map to 'Other' 
# Kleinwagon could be mapped to multiple targets, so we cannot assume 'other' . 
type_map ={
  'Cabriolet': 'Convertible / Roadster',
  'Coupé': 'Coupé',
  'Kleinwagen': 'Null',
  'Kombi': 'Station Wagon',
  'Kompaktvan / Minivan': 'Other',
  'Limousine': 'Saloon',
  'Pick-up': 'Other',
  'SUV / Geländewagen': 'SUV',
  'Sattelschlepper': 'Other',
  'Wohnkabine': 'Other',
  'nan': 'Null'}

normal_df['carType'] = normal_df['BodyTypeText'].map(type_map)
normal_df[ ['BodyTypeText', 'carType'] ]

,BodyTypeText,carType
0,Cabriolet,Convertible / Roadster
1,SUV / Geländewagen,SUV
2,Kombi,Station Wagon
3,Limousine,Saloon
4,Cabriolet,Convertible / Roadster
...,...,...
1148,Limousine,Saloon
1149,Coupé,Coupé
1150,SUV / Geländewagen,SUV
1151,Cabriolet,Convertible / Roadster


In [ ]:
# Mapping the colour fields to match the available values in the target data, translated into English
# Metallic colours are mapped to their non metallic equivalents. 
# Anthracite is assumed to be Gray
color_map = {'anthrazit':'Gray', 
             'anthrazit mét.':'Gray', 
             'beige':'Beige', 
             'beige mét.':'Beige', 
             'blau':'Blue',
             'blau mét.':'Blue', 
             'bordeaux':'Other',
             'bordeaux mét.':'Other',
             'braun':'Brown',
             'braun mét.':'Brown',
             'gelb':'Yellow', 
             'gelb mét.':'Yellow', 
             'gold':'Gold',
             'gold mét.':'Gold',
             'grau':'Gray', 
             'grau mét.':'Gray',
             'grün':'Green', 
             'grün mét.':'Green', 
             'orange':'Orange', 
             'orange mét.':'Orange', 
             'rot':'Red', 
             'rot mét.':'Red',
             'schwarz':'Black', 
             'schwarz mét.':'Black', 
             'silber':'Silver', 
             'silber mét.':'Silver', 
             'violett mét.':'Other',
             'weiss':'White',
             'weiss mét.':'White'}
normal_df['color'] = normal_df['BodyColorText'].map(color_map) 
normal_df[ ['BodyColorText', 'color'] ]

,BodyColorText,color
0,silber mét.,Silver
1,silber mét.,Silver
2,schwarz,Black
3,schwarz mét.,Black
4,schwarz mét.,Black
...,...,...
1148,grün mét.,Green
1149,schwarz,Black
1150,schwarz,Black
1151,silber mét.,Silver


In [ ]:
# Fix some differences in capitalisation of the 'MakeText' fields, to better match the target data. Avoid changing acronyms. 
for make in normal_df['MakeText'].unique():
  if make not in {'BMW', 'VW', 'MG', 'NSU', 'PGO', 'AGM', 'RUF' }:
    normal_df.loc[ normal_df['MakeText'] == make, 'make' ] = make.title()
  else: 
    normal_df.loc[ normal_df['MakeText'] == make, 'make' ] = make.upper()
normal_df['make'].unique()

array(['Mercedes-Benz', 'Audi', 'Saab', 'Porsche', 'Chevrolet', 'Matra',
       'BMW', 'Renault', 'Rolls-Royce', 'Alfa Romeo', 'Simca', 'Honda',
       'VW', 'Smart', 'Ferrari', 'Bmw-Alpina', 'Maserati', 'Toyota',
       'Lamborghini', 'Triumph', 'Talbot', 'Land Rover', 'Mclaren',
       'Jaguar', 'Bentley', 'Lancia', 'Daihatsu', 'Opel', 'Mazda', 'RUF',
       'Dodge', 'NSU', 'Volvo', 'Lotus', 'Aston Martin', 'Mitsubishi',
       'Nissan', 'Mini', 'Chrysler', 'Jeep', 'Lincoln', 'Hummer',
       'Ford (Usa)', 'De Tomaso', 'MG', 'Ford', 'Hyundai', 'Daewoo',
       'Daimler', 'Yes!', 'Citroen', 'Marcos', 'Trabant', 'Peugeot',
       'Fiat', 'Austin-Healey', 'Wiesmann', 'Bugatti', 'Giottiline',
       'Seat', 'Kaiser', 'Subaru', 'Autobianchi', 'Datsun', 'Delorean',
       'PGO', 'Harley-Davidson', 'Morgan', 'AGM', 'Lexus'], dtype=object)

In [ ]:
# Create a new column in the supplier data to integrate with the target column 'model_variant'. 
# This is done by taking the existing 'ModelTypeText' string and, if it appears, removing from it the 'ModelType' string. 
# The case where the ModelType is capitalized differently is also covered. 
for model in normal_df['ModelText'].unique():
  try:
    normal_df.loc[ normal_df['ModelText'] == model, 'model_variant'] = normal_df.loc[ normal_df['ModelText'] == model, 'ModelTypeText'].str.replace(model, '').str.replace(model.capitalize(), '')
  except: pass
normal_df['model'] = normal_df['ModelText']
normal_df.fillna('null', inplace = True)
normal_df[['ModelText', 'ModelTypeText', 'model_variant']].head()

,ModelText,ModelTypeText,model_variant
0,SLR,SLR McLaren,McLaren
1,ML 350,ML 350 Inspiration,Inspiration
2,S6,S6 Avant quattro 4.2,Avant quattro 4.2
3,9-3,9-3 2.0i-16 TS Aero,2.0i-16 TS Aero
4,911,911 Turbo Cabrio,Turbo Cabrio


In [ ]:
# Map the condition. We lose some information because the target set does not contain as many possible values
conditions_map = {
    'Neu':'New',
    'Occasion':'Used',
    'Oldtimer':'Other',
    'Vorführmodell':'Other'
}
# Make a new column to match the heading in the target data
normal_df['condition'] = normal_df['ConditionTypeText'].map(conditions_map)
normal_df[['ConditionTypeText', 'condition']].sample(10)

,ConditionTypeText,condition
1050,Occasion,Used
921,Occasion,Used
874,Neu,New
908,Occasion,Used
436,Occasion,Used
157,Occasion,Used
691,Occasion,Used
34,Occasion,Used
1040,Occasion,Used
584,Occasion,Used


In [ ]:
# Normalising the kilometers column to match, 'mileage' and adding 'mileage unit' (always km), renaming some columns to match target, filling empty values with 'null' as in target. 
normal_df['mileage'] = normal_df['Km']
normal_df['city'] = normal_df['City']
normal_df['mileage_unit'] = 'kilometers'
normal_df.fillna(value = 'null', inplace=True) 


In [ ]:
normal_df.head() # Let's take a look at the dataset with new normalised columns included. 

,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,BodyColorText,BodyTypeText,Ccm,City,Co2EmissionText,ConditionTypeText,ConsumptionRatingText,ConsumptionTotalText,Doors,DriveTypeText,FirstRegMonth,FirstRegYear,FuelTypeText,Hp,InteriorColorText,Km,Properties,Seats,TransmissionTypeText,carType,color,make,model,model_variant,condition,mileage,city,mileage_unit
0,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,silber mét.,Cabriolet,5439,Zuzwil,null,Occasion,null,null,2,Hinterradantrieb,10,2007,Benzin,626,schwarz,29800,"""Ab MFK""",2,Automatik-Getriebe,Convertible / Roadster,Silver,Mercedes-Benz,SLR,McLaren,Used,29800,Zuzwil,kilometers
1,MERCEDES-BENZ,ML 350 Inspiration,MERCEDES-BENZ ML 350 Inspiration,ML 350,ML 350 Inspiration,silber mét.,SUV / Geländewagen,3724,Zuzwil,305 g/km,Occasion,G,12.7 l/100km,5,Allrad,10,2003,Benzin,235,schwarz,126300,"""Ab MFK""",5,Automat,SUV,Silver,Mercedes-Benz,ML 350,Inspiration,Used,126300,Zuzwil,kilometers
2,AUDI,S6 Avant quattro 4.2,AUDI S6 Avant quattro 4.2,S6,S6 Avant quattro 4.2,schwarz,Kombi,4170,Zuzwil,353 g/km,Occasion,null,14.9 l/100km,5,Allrad,6,1997,Benzin,290,beige,91900,"""Ab MFK""",5,Automat,Station Wagon,Black,Audi,S6,Avant quattro 4.2,Used,91900,Zuzwil,kilometers
3,SAAB,9-3 2.0i-16 TS Aero,SAAB 9-3 2.0i-16 TS Aero,9-3,9-3 2.0i-16 TS Aero,schwarz mét.,Limousine,1985,Zuzwil,216 g/km,Occasion,G,9.0 l/100km,3,Vorderradantrieb,9,2001,Benzin,205,schwarz,134600,"""Ab MFK""",5,Schaltgetriebe manuell,Saloon,Black,Saab,9-3,2.0i-16 TS Aero,Used,134600,Zuzwil,kilometers
4,PORSCHE,911 Turbo Cabrio,PORSCHE 911 Turbo Cabrio,911,911 Turbo Cabrio,schwarz mét.,Cabriolet,3600,Zuzwil,309 g/km,Occasion,G,12.9 l/100km,2,Allrad,4,2008,Benzin,480,braun,35500,"""Ab MFK""",2,Schaltgetriebe manuell,Convertible / Roadster,Black,Porsche,911,Turbo Cabrio,Used,35500,Zuzwil,kilometers


In [ ]:
# Condensing the input data into a new set with only the relevant columns, ready for integration into the target set. 
integrated_df = normal_df[['carType', 'color', 'condition', 'city', 'make', 'mileage', 'mileage_unit', 'model', 'model_variant'  ]]
integrated_df.head()

,carType,color,condition,city,make,mileage,mileage_unit,model,model_variant
0,Convertible / Roadster,Silver,Used,Zuzwil,Mercedes-Benz,29800,kilometers,SLR,McLaren
1,SUV,Silver,Used,Zuzwil,Mercedes-Benz,126300,kilometers,ML 350,Inspiration
2,Station Wagon,Black,Used,Zuzwil,Audi,91900,kilometers,S6,Avant quattro 4.2
3,Saloon,Black,Used,Zuzwil,Saab,134600,kilometers,9-3,2.0i-16 TS Aero
4,Convertible / Roadster,Black,Used,Zuzwil,Porsche,35500,kilometers,911,Turbo Cabrio


In [ ]:
# Writing the dataframes into the 3 requested excel sheets
with pd.ExcelWriter('Onedot.xlsx') as writer:
  pre_pro_df.to_excel(writer, sheet_name='Pre-processing', index = False)
  normal_df.to_excel(writer, sheet_name='Normalising', index = False)
  integrated_df.to_excel(writer, sheet_name='Integrated', index = False)
